In [2]:
import requests
import json
import time
import os
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt #

In [3]:
#API key
api_key = "RGAPI-73eb964c-cf9a-494b-81f6-6c5141c32883"
gameName = "Slylio" #username
tagLine = "EUW" #suffixe

In [4]:
#Player unviveraly unique identifier
requestPUUID = requests.get(f"https://europe.api.riotgames.com/riot/account/v1/accounts/by-riot-id/{gameName}/{tagLine}?api_key={api_key}")
puuid = requestPUUID.json()["puuid"] 
print(puuid)

nICwUFHOH_TsQczqTIDUQioPzLXiN3fytgCmRxKhbq0puAX8nqzIEWAsPLP2prR7H3bQF3DOvJ4iyg


In [5]:
#Match history from PUUID 
nb_of_games = 2
games = requests.get(f"https://europe.api.riotgames.com/lol/match/v5/matches/by-puuid/{puuid}/ids?type=ranked&start=0&count={nb_of_games}&api_key={api_key}")
for game in games.json():
    print(game)

EUW1_6835115969
EUW1_6835054292


In [7]:
#Extract matchs data from match ids
matchs = games.json()
matchs_data=[]
data = pd.DataFrame()
cout=0
for matchId in matchs:
    cout+=1
    print("Match n°",cout,"/",nb_of_games,"\r",end="")
    match_data = requests.get(f"https://europe.api.riotgames.com/lol/match/v5/matches/{matchId}?api_key={api_key}")
    matchs_data.append(match_data.json())
    


In [12]:
def get_matchids_by_puuid(puuid, num, api_key):

    matchids_list = []


    api_url = 'https://europe.api.riotgames.com/lol/match/v5/matches/by-puuid/{}/ids?queue=420&type=ranked&start=0&count={}&api_key={}'.format(puuid,num, api_key)

    resp = requests.get(api_url)
    print(resp.status_code)
    if resp.status_code == 200: 
        pass
    elif resp.status_code == 429: #429 error is api cost issue
        while True: # while loop because of riot api cost
            if resp.status_code == 429: #429 error is api cost issue
                print('429 delay try 100 second') #approximate 110 second wait
                time.sleep(10)
                api_url = 'https://europe.api.riotgames.com/lol/match/v5/matches/by-puuid/{}/ids?queue=420&type=ranked&start=0&count={}&api_key={}'.format(puuid,num, api_key)
                resp = requests.get(api_url)
                print(resp.status_code)
            elif resp.status_code == 200:
                print('limit cost resolve')
                break        

    matchids = pd.DataFrame(resp.json(), columns = ['matchid']) 
    matchids_list.append(matchids) # Appending the DataFrame generated to the list "matchids_list"

    matchids_df = pd.concat(matchids_list) 
    
    return matchids_df

In [13]:
def get_match_info_by_matchId(match_ids, api_key):

    matchid_info_list = [] # List that will store the dataframes generated, for each match id in the for loop 

    for i in range(len(match_ids)):# len() get the numbers of rows of the match_ids parameter inserted. Then we use the range() function to create a sequence of number starting from 0 to number returned by len(). 
                            # With that we can interate trough each line in the match_ids dataframe (our function parameter) using the iloc function to get the matchid.

        api_ulr = 'https://europe.api.riotgames.com/lol/match/v5/matches/{}?api_key={}'.format(match_ids['matchid'].iloc[i], api_key) # here we use the iloc function as explained above to form the api link for each matchid
        resp = requests.get(api_ulr) # Sending a get request to the api_url
        print(resp.status_code) # Printing the status_code of the request

        if resp.status_code == 200: # Here we need to use if statment, because of the riot API rate limit which is 20 requests every 1 sec and 100 requests every 2 minutes. A 200 status_code means that the request was successful so it can move on.
            pass                        
        elif resp.status_code == 429: # A 429 means that we have exceeded the rate limit. To solve that we need to wait and then try again to get a successful request
            while True: # To do taht we use a while true loop, to execute the if, elif statment repeatdly until the status_code is 200. After that we use break statement to come out of this loop.
                if resp.status_code == 429: 
                    print('429 delay try 10 second') #approximate 110 second wait to work
                    time.sleep(10) # Sleep

                    api_url = 'https://europe.api.riotgames.com/lol/match/v5/matches/{}?api_key={}'.format(match_ids['matchid'].iloc[i], api_key) # Trying the request again
                    resp = requests.get(api_url)
                    print(resp.status_code)

                elif resp.status_code == 200:
                    print('limit cost resolve')
                    break        

        # Same process we did in the beginning             
        matchid_info = pd.json_normalize(resp.json()['info']['participants'])
        matchid_info_list.append(matchid_info) # Merging all the dataframes generated and then stored in the matchid_info_list with the concat() function
        
    matchid_info_df = pd.concat(matchid_info_list)

    return matchid_info_df

In [17]:
matchsid = get_matchids_by_puuid(puuid, 10, api_key)
matchid_info_df= get_match_info_by_matchId(matchsid, api_key)


200
200
200
200
200
200
200
200
200
200
200


In [1]:
matchid_info_df

NameError: name 'matchid_info_df' is not defined